In [1]:
## anaconda3 (Python 3.12.0) Kernel
import numpy as np

# pair trade packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from datetime import datetime
import numpy as np
from collections import defaultdict

# Load Pairs Data


In [2]:
def custom_date_parser(date_str):
    return datetime.strptime(date_str, '%d/%m/%Y')

# Load the dictionary from the pickle file
with open('pairsOutcome.pkl', 'rb') as file:
    pairsOutcome = pickle.load(file)

print("Dictionary loaded from pairsOutcome.pkl")


# Load stock data and get return 
tpxData = pd.read_csv('TPX_prices.csv', index_col=0, parse_dates=True, date_parser=custom_date_parser)
tpxData = tpxData.dropna(axis='columns')
return_df = (tpxData / tpxData.shift(1)) - 1

Dictionary loaded from pairsOutcome.pkl


/var/folders/h2/r7qn2m9n1zb6y_0q191gdqth0000gn/T/ipykernel_34700/3199335484.py:12: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  tpxData = pd.read_csv('TPX_prices.csv', index_col=0, parse_dates=True, date_parser=custom_date_parser)


# Get Pair Trade Portfolio
`pairsOutcome` already have TOPIX stocks with highest liquidity and are tested for stationarity over a 1 year window

Choose top 10 known pair trades by returns in the total dataset

In [3]:
working_pairs = [('5401 JP Equity', '8604 JP Equity'), ('6273 JP Equity', '9984 JP Equity'), ('8053 JP Equity', '8058 JP Equity'), ('7733 JP Equity', '9613 JP Equity'), ('4684 JP Equity', '7832 JP Equity'), ('6762 JP Equity', '6857 JP Equity'), ('9020 JP Equity', '9022 JP Equity'), ('7267 JP Equity', '8306 JP Equity'), ('8308 JP Equity', '8802 JP Equity'), ('4901 JP Equity', '6702 JP Equity'), ('6503 JP Equity', '7269 JP Equity'), ('7267 JP Equity', '8801 JP Equity'), ('4519 JP Equity', '7532 JP Equity'), ('6988 JP Equity', '7267 JP Equity'), ('6326 JP Equity', '6954 JP Equity'), ('6752 JP Equity', '8604 JP Equity'), ('4901 JP Equity', '9613 JP Equity')]
top_keys = [f"{pair[0]} {pair[1]}" for pair in working_pairs]

In [4]:
# Print the top 10 performing trades
print("Top 10 performing trades:")
for i, key in enumerate(top_keys, 1):
    print(f"{i}. Key: {key}, Value: {pairsOutcome[key].cumpnl.iloc[-2]}")

Top 10 performing trades:
1. Key: 5401 JP Equity 8604 JP Equity, Value: 0.008934753682416807
2. Key: 6273 JP Equity 9984 JP Equity, Value: 0.9917734886387003
3. Key: 8053 JP Equity 8058 JP Equity, Value: 0.5183664464524095
4. Key: 7733 JP Equity 9613 JP Equity, Value: 0.33979225570551896
5. Key: 4684 JP Equity 7832 JP Equity, Value: 0.8866951703468606
6. Key: 6762 JP Equity 6857 JP Equity, Value: -0.6701447016767165
7. Key: 9020 JP Equity 9022 JP Equity, Value: 0.30716236746749637
8. Key: 7267 JP Equity 8306 JP Equity, Value: 1.1628735722945633
9. Key: 8308 JP Equity 8802 JP Equity, Value: 0.4326318755024665
10. Key: 4901 JP Equity 6702 JP Equity, Value: -0.3387706704830611
11. Key: 6503 JP Equity 7269 JP Equity, Value: 1.332416127365959
12. Key: 7267 JP Equity 8801 JP Equity, Value: 0.6442746073863653
13. Key: 4519 JP Equity 7532 JP Equity, Value: 1.138329913797488
14. Key: 6988 JP Equity 7267 JP Equity, Value: 0.6453419709959551
15. Key: 6326 JP Equity 6954 JP Equity, Value: 1.193819

In [5]:
## Get pair stock data
def custom_date_parser(date_str):
    return datetime.strptime(date_str, '%d/%m/%Y')
valid = pd.read_csv('validPairs5.csv', 
                    index_col=0, 
                    parse_dates=True, 
                    date_parser=custom_date_parser)
## get list of pair stocks
validPairsList = [
    [item.strip() + ' Equity' for item in pair.split('Equity') if item.strip()]
    for pair in top_keys
]

/var/folders/h2/r7qn2m9n1zb6y_0q191gdqth0000gn/T/ipykernel_34700/2263820838.py:4: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  valid = pd.read_csv('validPairs5.csv',


In [6]:
rollingWindow = 262
cutLossSd = 2

In [7]:
for pair in validPairsList:
    df = pd.DataFrame()

    #Calculate Standard Deviations
    df['spread'] = valid[f'spread_{pair[0]}_{pair[1]}']
    df['mid'] =  df['spread'].rolling(rollingWindow).mean()
    df['1sd high'] = df['spread'].rolling(rollingWindow).mean() + df['spread'].rolling(rollingWindow).std()
    df['1sd low'] = df['spread'].rolling(rollingWindow).mean() - df['spread'].rolling(rollingWindow).std()
    df['2sd high'] = df['spread'].rolling(rollingWindow).mean() + df['spread'].rolling(rollingWindow).std() * cutLossSd
    df['2sd low'] = df['spread'].rolling(rollingWindow).mean() - df['spread'].rolling(rollingWindow).std() * cutLossSd
    df['position'] = 0

    df.loc[(df['spread'] > df['1sd high']) & (df['spread'] < df['2sd high']), 'position'] = -1
    df.loc[(df['spread']< df['1sd low']) & (df['spread'] > df['2sd low']), 'position'] = 1

    #Calculate PnL
    df[f'{pair[0]} position'] = df['position']
    df[f'{pair[1]} position'] = df['position'] * -1
    df['dailypnl'] = df[f'{pair[1]} position']*return_df[f'{pair[1]}'].shift(-1) + df[f'{pair[0]} position']*return_df[f'{pair[0]}'].shift(-1)
    df['cumpnl'] = df['dailypnl'].cumsum()

    pairsOutcome[f'{pair[0]} {pair[1]}'] = df

## Make indicators and spread stationary around 0
Deduct the mean from all values to translate to 0 axis

In [8]:
workingPairOutcome = {}

for pair in top_keys:
    dummy_df = pairsOutcome[top_keys[0]].iloc[::,:6]
    dummy_df = dummy_df.subtract(dummy_df['mid'], axis=0).drop(columns=['mid']) # centre spread and SD
    dummy_df = dummy_df.div(dummy_df['2sd high']-dummy_df['1sd high'],axis=0)   # express SD as integers, give spread as propotionate
    dummy_df['2sd_high_boolean'] = (dummy_df['spread']>dummy_df['2sd high']).astype(int)
    dummy_df['1sd_high_boolean'] = (dummy_df['spread']>dummy_df['1sd high']).astype(int)
    dummy_df['0sd_high_boolean'] = (dummy_df['spread']>0).astype(int)
    dummy_df['0sd_low_boolean']  = (dummy_df['spread']<0).astype(int)
    dummy_df['1sd_low_boolean']  = (dummy_df['spread']<dummy_df['1sd low'] ).astype(int)
    dummy_df['2sd_low_boolean']  = (dummy_df['spread']<dummy_df['2sd low'] ).astype(int)
    dummy_df = dummy_df.drop(columns=['spread','1sd high', '1sd low', '2sd high', '2sd low'])
    workingPairOutcome[pair] = dummy_df.to_numpy()

In [9]:
workingPairOutcome[top_keys[5]][-5:]     # spread is not a proportion and direction of SD

array([[0, 0, 0, 1, 1, 1],
       [0, 0, 0, 1, 1, 0],
       [0, 0, 0, 1, 1, 0],
       [0, 0, 0, 1, 1, 0],
       [0, 0, 0, 1, 1, 0]])

# Machine Learning Challenge

## Background
Initial evaluation of the baseline portfolio shows that draw downs are small. Originally team had the idea of using Machine Learning to optimise for sizing of these pair trades. However since there was no significant drawdowns the returns are linearly increasing with investment sizing i.e. greater nominal investment in the the pair trade the proportionate increase in returns without realising significant drawdown risk.

Instead of optimising for sizing, we can explore Machine Learning in terms of strategy on this stationary dataset. Whereas our prescribed strategy is to enter at +/- 1 std dev, exit at 0 with +/- 2 std dev stop loss. These are only suggestions and arbitrary levels.

With Machine Learning, we can discover if it will uncover the mean reverting nature and recommend another threshhold. We use Q Learner to understand state space with the same spread, mid, std dev parameters as the baseline.

### Q Value table

In [10]:
class PairTradeEnv1:
    def __init__(self):
        # Initialize environment variables and parameters
        self.num_actions = 3
        self.num_states = 2**6
        self.earliest_step = 261  # hot start
        self.last_step = 2868

        self.state = np.zeros(4)
        self.current_step = self.earliest_step

    def reset(self,pair_idx):
        # Reset the environment to its initial state
        self.current_step = self.earliest_step
        self.state = workingPairOutcome[top_keys[pair_idx]][self.current_step]
        return self.state

    def step(self, action,pair_idx):
        # Advance the time step
        self.current_step += 1
        next_state = workingPairOutcome[top_keys[pair_idx]][self.current_step]
        reward = self.calculate_reward(action,pair_idx)
        done = self.current_step >= self.last_step
        return next_state, reward, done
    
    def calculate_reward(self, position,pair_idx):
        """
        Give one _previous_ day's return
        Input:
            position: position for idx (current step)
            idx: usually current timestp 
            pair: tuple of tpx stock
        Output:
            dailypnl
        """
        pair = validPairsList[pair_idx]
        # position = position_vector @ np.array([-1,0,1])
        position_0 = position
        position_1 = position * -1
        ## return_df gives the return for the previous day for the given idx
        dailypnl = position_0*return_df[f'{pair[0]}'].iloc[self.current_step] + position_1*return_df[f'{pair[1]}'].iloc[self.current_step] 

        return dailypnl

class Agent:
    def __init__(self, num_states, num_actions, alpha=0.3, gamma=0.9, epsilon=0.3):
        self.num_actions = num_actions
        self.num_states = num_states
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon

        self.action_to_index = {-1: 0, 0: 1, 1: 2}
        self.index_to_action = {0: -1, 1: 0, 2: 1}
        self.Q = np.zeros((num_states, num_actions))
        
        # State visitation frequency
        self.state_counts = np.zeros(num_states)

    def b_to_d_state(self, binary_state):
        decimal_index = sum(binary_state[i] * 2**(5 - i) for i in range(6))
        return decimal_index

    def choose_action(self, state):
        if np.random.rand() < self.epsilon:
            action_index = np.random.randint(self.num_actions)
        else:
            state_index = self.b_to_d_state(state)
            action_index = np.argmax(self.Q[state_index])
        return self.index_to_action[action_index]

    def update_Q(self, state, action, reward, next_state):
        state_index = self.b_to_d_state(state)
        next_state_index = self.b_to_d_state(next_state)
        self.Q[state_index, self.action_to_index[action]] += self.alpha * (
            reward + self.gamma * np.max(self.Q[next_state_index]) - self.Q[state_index, self.action_to_index[action]]
        )
        self.state_counts[state_index] += 1

    def learn(self, num_episodes, env):
        for episode in range(num_episodes):
            for pair_idx in range(10):
                # Sample initial state with bias toward underrepresented states
                state_probabilities = 1 / (self.state_counts + 1)
                state_probabilities /= state_probabilities.sum()
                sampled_state_index = np.random.choice(self.num_states, p=state_probabilities)

                state = env.reset(pair_idx)
                done = False

                while not done:
                    action = self.choose_action(state)
                    next_state, reward, done = env.step(action, pair_idx)
                    reward *= 10
                    self.update_Q(state, action, reward, next_state)
                    state = next_state

                


num_episodes = 100

env = PairTradeEnv1()
agent = Agent(num_states=env.num_states, num_actions=env.num_actions)
agent.learn(num_episodes, env=env)

In [11]:
binary_labels = [f"{i:06b}" for i in range(64)]
df = pd.DataFrame(agent.Q, index=binary_labels, columns=[-1, 0, 1])
df = df[(df != 0).any(axis=1)]

In [12]:
df['mean_reversion'] = [0,1,0,0,-1,0]
df['position'] = df[[-1, 0, 1]].idxmax(axis=1)
df

,-1,0,1,mean_reversion,position
000100,0.084522,0.056134,-0.004717,0,-1
000110,0.118823,0.093761,0.131606,1,1
000111,0.157484,0.103588,0.063113,0,-1
001000,0.073366,0.076011,0.064393,0,0
011000,0.005333,0.055637,0.019247,-1,0
111000,0.009486,0.069012,-0.007259,0,0


table is taking a long time to generalise. using one pair is not good enough to get any poistioning as everything is flat. only after adding all 10 pairs then performance churns out shorting on 1 SD high cross.

In [13]:
dummy_df.columns

Index(['2sd_high_boolean', '1sd_high_boolean', '0sd_high_boolean',
       '0sd_low_boolean', '1sd_low_boolean', '2sd_low_boolean'],
      dtype='object')

In [14]:
# Get baseline results
t_pair = validPairsList[0]
max_steps_per_episode = 3000
agent.epsilon = 0

def get_baseline(env, max_steps_per_episode, pair_idx):
    agent.epsilon = 0
    env.reset(pair_idx)
    total_reward = 0
    current_step = 261
    env.current_step = current_step
    env.last_step = 2868

    for step in range(max_steps_per_episode):
        action = pairsOutcome[top_keys[pair_idx]].iloc[env.current_step]['position']
        _, reward, done = env.step(action,pair_idx)
        total_reward += reward

        if done:
            break

    print(f"Baseline {t_pair}, Total Reward: {total_reward}, step {step}")

get_baseline(env, 3000, pair_idx=0)

Baseline ['5401 JP Equity', '8604 JP Equity'], Total Reward: 0.365425120007449, step 2606


In [15]:
import numpy as np
from collections import Counter

array = workingPairOutcome['5401 JP Equity 8604 JP Equity']
# Convert rows to tuples (hashable type) for counting
row_tuples = map(tuple, array)

# Count unique rows
row_counts = Counter(row_tuples)
q_policy_actions = df.position.values
ls_state = []
# Print results
for row, count in row_counts.items():
    ls_state.append(row)
    print(f"Row: {row}, Count: {count}")

ls_state = ls_state[1:]
ls_state.sort()
dict_policy_actions = dict(zip(ls_state, q_policy_actions))
dict_policy_actions[(0, 0, 0, 0, 0, 0)] = 0
dict_policy_actions

Row: (0, 0, 0, 0, 0, 0), Count: 261
Row: (0, 0, 1, 0, 0, 0), Count: 712
Row: (0, 1, 1, 0, 0, 0), Count: 703
Row: (1, 1, 1, 0, 0, 0), Count: 162
Row: (0, 0, 0, 1, 0, 0), Count: 446
Row: (0, 0, 0, 1, 1, 0), Count: 470
Row: (0, 0, 0, 1, 1, 1), Count: 225


{(0, 0, 0, 1, 0, 0): -1,
 (0, 0, 0, 1, 1, 0): 1,
 (0, 0, 0, 1, 1, 1): -1,
 (0, 0, 1, 0, 0, 0): 0,
 (0, 1, 1, 0, 0, 0): 0,
 (1, 1, 1, 0, 0, 0): 0,
 (0, 0, 0, 0, 0, 0): 0}

In [16]:
workingPairOutcome['5401 JP Equity 8604 JP Equity']

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 1, 1, 0],
       [0, 0, 0, 1, 1, 0],
       [0, 0, 0, 1, 1, 0]])

In [18]:
workingPairQresults = {}

for pair_idx in range(len(top_keys)):
    df = pd.DataFrame(workingPairOutcome[top_keys[pair_idx]], columns=dummy_df.columns)

    # Assign policy values using the dictionary
    df['position'] = df.apply(lambda row: dict_policy_actions.get(tuple(row), np.nan), axis=1)
    df[df.isna().any(axis=1)]
    pair = validPairsList[pair_idx]
    #Calculate PnL
    df[f'{pair[0]} position'] = df['position']
    df[f'{pair[1]} position'] = df['position'] * -1
    df['dailypnl'] = df[f'{pair[1]} position'].values*return_df[f'{pair[1]}'].shift(-1).values \
                    + df[f'{pair[0]} position'].values*return_df[f'{pair[0]}'].shift(-1).values
    df['cumpnl'] = df['dailypnl'].cumsum()
    workingPairQresults[top_keys[pair_idx]] = df

In [19]:
total_q_return = 0
total_train_q_return = 0
total_test_q_return = 0

for pair in top_keys:
    total_q_return += workingPairQresults[pair]['cumpnl'].iloc[-2]
    total_train_q_return += workingPairQresults[pair]['cumpnl'].iloc[2868-2]


print(f"total return \t\t{total_q_return/len(top_keys):.5f}")
print(f"total train return \t{total_train_q_return/len(top_keys):.5f}")
print(f"total test return \t{(total_q_return - total_train_q_return)/len(top_keys):.5f}")

total return 		0.20227
total train return 	0.18891
total test return 	0.01336
